In [1]:
# 필요한 라이브러리 불러오기
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import json
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
# 정보를 저장할 데이터프레임 생성
columns = ['name', 'brand', 'price', 'discount_rate', 'final_price', 'score_average', 'score_count', 'user_name', 'review_title', 'review', 'score']
df = pd.DataFrame(columns=columns)
df

,name,brand,price,discount_rate,final_price,score_average,score_count,user_name,review_title,review,score


In [23]:
# 크롬 드라이버 설정
options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 시작 페이지 접속(카테고리별 페이지)
driver.get('https://www.coupang.com/np/categories/486249')
time.sleep(5)

# 페이지 이동
for k in range(2, 3):
    if k >= 2:
        page_element = driver.find_element(By.XPATH, f"/html/body/div[3]/section/form/div/div/div[1]/div/div[4]/div/a[{k+1}]")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        driver.execute_script("arguments[0].scrollIntoView(true);", page_element)
        page_element.click()
        time.sleep(5)
        
    # 상품 클릭 후 정보 수집, 뒤로가기 반복    
    for i in range(21, 61):
        
        # 상품 클릭
        product_element = driver.find_element(By.XPATH, f"/html/body/div[3]/section/form/div/div/div[1]/div/ul/li[{i}]/a/dl")
        driver.execute_script("arguments[0].scrollIntoView(true);", product_element)
        product_element.click()
        time.sleep(5)
    
        # 상품 정보 수집(이름, 브랜드, 가격, 할인율)
        item_info = []
        
        try: name = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[1]/div[3]/div[3]/h1").text
        except: 
            try: name = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[1]/div[3]/div[4]/h1").text
            except: name = " "
            
        try: brand = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[1]/div[3]/a").text
        except: brand = " "
            
        try: price = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[1]/div[3]/div[5]/div[1]/div/div[1]/span[2]").text
        except: price = " "
            
        try: discount_rate = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[1]/div[3]/div[5]/div[1]/div/div[1]/span[1]").text
        except: discount_rate = " "
            
        try: final_price = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[1]/div[3]/div[5]/div[1]/div/div[2]/span/strong").text
        except: final_price = " "
        
        # 상품평 탭 클릭
        time.sleep(1)
        score_element = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[2]/div[8]/ul[1]/li[2]")
        driver.execute_script("arguments[0].scrollIntoView(true);", score_element)
        score_element.click()
        time.sleep(5)
    
        # 평균별점, 리뷰개수
        try: score_average = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[2]/div[8]/ul[2]/li[2]/div/div[4]/section[1]/div[1]/div[1]/div").get_attribute('data-rating')
        except: score_average = " "
        try: score_count = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[2]/div[8]/ul[2]/li[2]/div/div[4]/section[1]/div[1]/div[2]").text
        except: score_count = " "
            
        item_info.extend([name, brand, price, discount_rate, final_price, score_average, score_count])   
        
        # 드롭다운(별점) 선택
        for z in range(1, 6):
            back_element = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[2]/div[8]/ul[2]/li[2]/div/h4")
            driver.execute_script("arguments[0].scrollIntoView(true);", back_element) # 위치 초기화
            time.sleep(1)
            try:
                dropbox_element = driver.find_element(By.XPATH, "/html/body/div[2]/section/div[2]/div[2]/div[8]/ul[2]/li[2]/div/div[6]/section[2]/div[3]")
                dropbox_element.click()
            except:
                continue
            time.sleep(3)
            option_element = driver.find_element(By.XPATH, f'//li[@data-rating="{z}"]')
            driver.execute_script("arguments[0].scrollIntoView(true);", option_element)
            option_element.click()
            time.sleep(3)
            
            # 사용자 반응 수집
            for j in range(1, 6):
                review_info = []
        
                # 사용자 아이디
                try:
                    user_element = driver.find_element(By.XPATH, f"/html/body/div[2]/section/div[2]/div[2]/div[8]/ul[2]/li[2]/div/div[6]/section[4]/article[{j}]/div[1]/div[2]/span")
                    driver.execute_script("arguments[0].scrollIntoView(true);", user_element)
                    user_name = user_element.text
                except: user_name = " "
        
                # 리뷰 제목
                try:
                    review_title_element = driver.find_element(By.XPATH, f"/html/body/div[2]/section/div[2]/div[2]/div[8]/ul[2]/li[2]/div/div[6]/section[4]/article[{j}]/div[3]")
                    driver.execute_script("arguments[0].scrollIntoView(true);", review_title_element)
                    review_title = review_title_element.text
                except: review_title = " "
        
                # 리뷰 내용
                try:
                    review_text_element = driver.find_element(By.XPATH, f"/html/body/div[2]/section/div[2]/div[2]/div[8]/ul[2]/li[2]/div/div[6]/section[4]/article[{j}]/div[4]/div")
                    driver.execute_script("arguments[0].scrollIntoView(true);", review_text_element)
                    review = review_text_element.text
                except: review = " "
            
                # 별점
                score = z
        
                # 데이터프레임에 저장
                review_info.extend([user_name, review_title, review, score])
                df.loc[len(df)] = item_info + review_info
        print(f'상품 {i} 추가 완료')
        driver.back()

상품 21 추가 완료
상품 22 추가 완료
상품 23 추가 완료
상품 24 추가 완료


In [10]:
df.shape

(2000, 11)

In [11]:
df.to_csv("./lotion_1_80.csv", encoding='"utf-8-sig"')